In [14]:
import pandas as pd
import re
import json
import time
import requests

In [5]:
df = pd.read_excel('addresses.xlsx')

In [6]:
df.head(2)

,Город отправки,Город доставки,Срок доставки,0-30 кг,+1 кг,100 кг,+1 кг.1,200 кг,+1 кг.2,500 кг,+1 кг.3
0,"г. Иркутск, ул. Розы Люксембург, 198 В","г Владивосток, ул Фадеева 47 Д",10-12,"1608,00","24,93","3352,92","20,14","5366,64","18,13","10806,60","16,32"
1,"г. Иркутск, ул. Розы Люксембург, 198 В","г Магадан, ул Пушкина, 11/11",31-33,"2046,00","39,62","4819,32","30,71","7890,24","27,12","16026,60","24,41"


In [7]:
fca = df['Город отправки'].unique().tolist() #first column addresses
sca = df['Город доставки'].unique().tolist() #second column addresses

In [8]:
all_addresses = fca + sca

In [9]:
yandex_token = 'YOUR YANDEX TOKEN'

In [10]:
mapbox_token='YOUR MAPBOX TOKEN'

In [11]:
def searchCoordinates(address):
    lat,lon = '0','0'
    preprocLine = re.sub(',\s*','+', address)
    request = 'https://geocode-maps.yandex.ru/1.x/?apikey='+yandex_token+'&format=json&geocode='+preprocLine+'&lang=ru_RU'
    r = requests.get(request)
    rec = json.loads(r.text)
    coordinates = rec['response']['GeoObjectCollection']['featureMember'][0]['GeoObject']['Point']['pos'].split(' ')
    lat = coordinates[1]
    lon = coordinates[0]
    return [lat,lon]

In [12]:
addresses_coordinates = {}

In [15]:
for address in all_addresses:
    addresses_coordinates[address] = searchCoordinates(address)

In [16]:
addresses_coordinates

{'г. Иркутск, ул. Розы Люксембург, 198 В': ['52.362802', '104.151626'],
 'г. Чита, ул. Читинская 23А': ['52.051578', '113.48682'],
 'г. Нижний Новгород, проезд Восточный, 11': ['56.251552', '43.91682'],
 'г. Пермь, ул. Уральская 81 а': ['58.027353', '56.296485'],
 'г Владивосток, ул Фадеева 47 Д': ['43.102892', '131.969172'],
 'г Магадан, ул Пушкина, 11/11': ['59.568392', '150.800762'],
 'г. Самара, Заводское шоссе, 20': ['53.201121', '50.259698'],
 'г. Волгоград, ул. Электролесовская, д.55': ['48.647268', '44.440951'],
 'г. Санкт-Петербург, Дальневосточный пр. 73 А.': ['59.876386', '30.482506'],
 'Калининградская обл., п. Дорожное, Датский проезд, д.3': ['54.642212',
  '20.520414'],
 'г. Новосибирск ул Н.Данченко 120/2': ['54.974313', '82.905859'],
 'г. Омск, ул. 22 Апреля, 19/1': ['55.040972', '73.277958'],
 'г.Екатеринбург, Эскадронная, 43': ['56.765759', '60.587818'],
 'г. Тюмень, ул. 50 лет Октября, 217а, склад 3/1': ['57.153653', '65.56673'],
 'г. Воронеж, ул. 9 января 302а': ['5

In [17]:
def search_distance(address1, address2):
    coordinates1 = list(reversed(addresses_coordinates[address1]))
    coordinates2 = list(reversed(addresses_coordinates[address2]))
    long_lat1 = ','.join(coordinates1)
    long_lat2 = ','.join(coordinates2)
    url = "https://api.mapbox.com/directions/v5/mapbox/driving-traffic/{};{}?access_token={}".format(long_lat1, long_lat2, mapbox_token)
    distance = 0
    try:
        t = requests.get(url, timeout=10)
        rec = json.loads(t.text)
        distance = float(rec['routes'][0]['distance'])/1000
    except Exception as error:
        print(str(error))
        print(rec)
    finally:
        return distance

In [18]:
with open('res_file.csv','w', encoding='cp1251') as file:
    file.write('Address1;Address2;Distance\n')
    for index,row in df.iterrows():
        distance = search_distance(row['Город отправки'],row['Город доставки']) 
        result_string = "{0};{1};{2}\n".format(row['Город отправки'],
                                                   row['Город доставки'],
                                                   distance)
        file.write(result_string)